In [1]:
import torch
import numpy as np
from models.training import easyTrainer, weights_to_dataset
from models.nODE import nODE, make_nODE_from_parameters
import torch
import torch.nn.functional as F
from torch.utils.data import Dataset
from torchdiffeq import odeint
from data_creation import create_dataset

In [2]:
x_train, x_noise, y_train, param = create_dataset(2,1,100)

In [3]:
class param_classifier(torch.nn.Module):
    def __init__(self, input_size, ode_dim, layers_size=[10, 10], device= torch.device('cuda' if torch.cuda.is_available() else 'cpu')):
        super().__init__()

        self.layers_size = layers_size
        self.input_size = input_size
        self.output_size = 2 * ode_dim ** 2 + 3 * ode_dim
        # print(self.output_size)
        self.ode_dim = ode_dim

        self.num_layers = len(layers_size) + 1
        self.device = device

        # initialise linear layers for classification block
        self.linears = torch.nn.ModuleList([torch.nn.Linear(input_size, layers_size[0])])
        self.linears.extend(
            [torch.nn.Linear(layers_size[i - 1], layers_size[i]) for i in range(1, self.num_layers - 1)])
        self.linears.append(torch.nn.Linear(layers_size[-1], self.output_size))
        return
    

    def forward_integration(self, x, parameter, integration_time=None):
        if integration_time is None:
            time_intervals = torch.tensor([0., 1.])
        else:
            time_intervals = torch.tensor(integration_time)
        integration_interval = torch.tensor([0,1.]).float().type_as(x)
        
        dt = 0.01
        out = odeint(lambda t, x : self.right_hand_side(t, x, parameter), x, integration_interval, method='euler', options={'step_size': dt})
        return out[-1, :]
    
    def get_adjacency(self,parameter):
        Win = torch.zeros(self.ode_dim,self.ode_dim).to(self.device)
        Wout = torch.zeros(self.ode_dim,self.ode_dim).to(self.device)

        k=0

        for i in range(0,self.ode_dim):
            for j in range(0,self.ode_dim):
                Win[i][j] = parameter[k]
                k += 1

        for i in range(0,self.ode_dim):
            for j in range(0,self.ode_dim):
                Wout[i][j] = parameter[k]
                k += 1
        
        A = Wout.matmul(Win).flatten()
        return A
               

    def right_hand_side(self, t, x, parameter):
        Win = torch.zeros(self.ode_dim,self.ode_dim).to(self.device)
        Wout = torch.zeros(self.ode_dim,self.ode_dim).to(self.device)
        bin = torch.zeros(self.ode_dim).to(self.device)
        bout = torch.zeros(self.ode_dim).to(self.device)
        gamma = torch.zeros(self.ode_dim).to(self.device)

        k=0

        for i in range(0,self.ode_dim):
            for j in range(0,self.ode_dim):
                Win[i][j] = parameter[k]
                k += 1

        for i in range(0,self.ode_dim):
            for j in range(0,self.ode_dim):
                Wout[i][j] = parameter[k]
                k += 1
        
        for i in range(0,self.ode_dim):
            bin[i] = parameter[k]
            k += 1

        for i in range(0,self.ode_dim):
            bout[i] = parameter[k]
            k += 1

        for i in range(0,self.ode_dim):
            gamma[i] = parameter[k]
            k += 1

        k=0
       
        out = x.matmul(Win.t()) + bin.t()
        out = out.matmul(Wout.t()) + bout.t()
        out = x.matmul(gamma) + out
        return out

    # forward pass of NN (both classifier and neural ODE)
    def forward(self, data):
        x = data
        for i in range(0, self.num_layers):
            x = self.linears[i](x)
            if i < self.num_layers-1:
                x = F.relu(x)

        # here x denote the estimated parameters for the ODE
        # x = self.linears[len(self.layers_size) - 1](x)

        return x

In [4]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

classifier = param_classifier(8, 2).to(device)

x = x_train[0][0].to(device)

out = classifier(x)

u0 = x[:2]

print(out)

ut_hat = classifier.forward_integration(u0, out)

print(ut_hat)



tensor([-0.4121,  0.3291, -0.1990, -0.0389, -0.0380,  0.0520, -0.0045, -0.2192,
        -0.0663,  0.1270,  0.1117,  0.6728, -0.1532, -0.2037], device='cuda:0',
       grad_fn=<AddBackward0>)
tensor([0.1485, 1.2627], device='cuda:0', grad_fn=<SliceBackward0>)


In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

classifier = param_classifier(8, 2).to(device)
ode_dim = 2
integration_time = 1
# node = nODE(ode_dim, architecture='both', time_interval=[0, integration_time]).to(device)
# # replaced by classifier.neuralODE
loss_fn = torch.nn.MSELoss()

optimizer = torch.optim.Adam(classifier.parameters(), lr=1e-2)
"""
for name, param in classifier.named_parameters():
    print(name, param.requires_grad)

for name, param in node.named_parameters():
    print(name, param.requires_grad)
"""
for epoch in range(100):
    optimizer.zero_grad()
    loss = 0

    for i in range(0, len(x_train)):
        x = x_train[i][0].to(device)
        y = y_train[i][0].to(device)

        # get necessary inputs from data
        classifier_inp = x

        # get u_0, u_T from data
        u0 = classifier_inp[:2]
        ut = y[:2]

        # get true adjacency from data
        A = y[2:]

        # estimate parameters using classifier network
        p = classifier(classifier_inp)

        # integrate ODE
        ut_hat = classifier.forward_integration(u0, p)

        # get adjacency matrix
        A_hat = classifier.get_adjacency(p)

        # add to loss: error between found solution at time t and true solution
        loss = loss + loss_fn(ut_hat.float(), ut.float())

        # add to loss: difference between estimated network and true network
        loss = loss + loss_fn(A_hat.float(),A.float())

        loss += loss_fn(p, 0*p)
    print('Epoch ' + str(epoch))
    print(loss)

    # backward propagation
    loss.backward(retain_graph=True)
    for name, param in classifier.named_parameters():
        if param.grad is None:
            print(f"Gradient for {name} is None")
        #else:
        #    # print(f"Gradient for {name}: {param.grad}")
    optimizer.step()
    # print(list(classifier.parameters())[0].grad)


Epoch 0
tensor(128.6659, device='cuda:0', grad_fn=<AddBackward0>)
Epoch 1
tensor(119.5470, device='cuda:0', grad_fn=<AddBackward0>)
Epoch 2
tensor(113.2022, device='cuda:0', grad_fn=<AddBackward0>)
Epoch 3
tensor(108.0327, device='cuda:0', grad_fn=<AddBackward0>)


KeyboardInterrupt: 

In [7]:
correct = 0
total = 0
for data_point in range(0, len(x_train)):
    x = x_train[i][0].to(device)
    out = classifier(x)

    print(out)

    # guess = out.cpu() > 0.7
    # print(guess)
    # print(data_point[1])
    # if sum(abs(guess.float()-data_point[1])) < 0.001:
    #     correct += 1
    # total += 1

print(correct/total)

tensor([0.0000, 0.0000, 0.0000, 0.0150, 0.0835, 0.0896, 0.0000, 0.0323, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0369], device='cuda:0',
       grad_fn=<ReluBackward0>)
tensor([0.0000, 0.0000, 0.0000, 0.0150, 0.0835, 0.0896, 0.0000, 0.0323, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0369], device='cuda:0',
       grad_fn=<ReluBackward0>)
tensor([0.0000, 0.0000, 0.0000, 0.0150, 0.0835, 0.0896, 0.0000, 0.0323, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0369], device='cuda:0',
       grad_fn=<ReluBackward0>)
tensor([0.0000, 0.0000, 0.0000, 0.0150, 0.0835, 0.0896, 0.0000, 0.0323, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0369], device='cuda:0',
       grad_fn=<ReluBackward0>)
tensor([0.0000, 0.0000, 0.0000, 0.0150, 0.0835, 0.0896, 0.0000, 0.0323, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0369], device='cuda:0',
       grad_fn=<ReluBackward0>)
tensor([0.0000, 0.0000, 0.0000, 0.0150, 0.0835, 0.0896, 0.0000, 0.0323, 0.0000,
        0.0000, 0.0000, 0.0000

ZeroDivisionError: division by zero